In [45]:
#!/usr/bin/env python3
import time
import numpy as np
from math import *
import matplotlib.pyplot as plt
from scipy import interpolate

In [46]:
#urx for ur10 and gripper
import urx
import math3d as m3d
from urx.robotiq_two_finger_gripper import Robotiq_Two_Finger_Gripper
ur = urx.Robot("192.168.1.102")
gripper = Robotiq_Two_Finger_Gripper(ur)
ur.set_tcp((0, 0, 0, 0, 0, 0))
tcp_acc=0.05
tcp_vel=0.05
time.sleep(0.2)
pose = ur.get_pose()
print(pose)

<Transform:
<Orientation: 
array([[ 9.99999990e-01, -1.37534017e-04,  4.26748796e-05],
       [-1.37546758e-04, -9.99999946e-01,  2.98697228e-04],
       [ 4.26337963e-05, -2.98703095e-04, -9.99999954e-01]])>
<Vector: (-0.08983, 0.48505, 0.36530)>
>


In [2]:
import palm

finding an odrive...
Odrive Voltage: 13.302319526672363


## start manipulation

In [52]:
### init gripper
gripper.gripper_activate()
time.sleep(1)
ur.set_tcp((0, 0, 0.295, 0, 0, 0)) #upper jaw's tip to tool0 29.5mm
int_list=np.array([0,170, 175, 180, 185, 190, 195, 200, 205, 210, 215,250])#
distance_list= np.array([130,31,28,25,22,19,15,13,9,7,4,0])/1000.0 
func_from_distance_to_int = interpolate.interp1d(distance_list, int_list, kind = 3)
center_width = 0.057 #jawtip at center of gripper:0.057
gripper.gripper_action(int(func_from_distance_to_int(center_width+0.02))) 

In [53]:
### init palm
palm.enable_motor()
# write_angle(inverse_kinematics((15,150)))
palm.goto((15,150))

enabling motor control...


#### pick object

In [50]:
### go pick pose
# print(ur.getl())
ur.movel((-0.09, 0.4865, 0.005, pi, 0, 0), tcp_acc, tcp_vel)
### close gripper and palm
grasp_width = center_width+0.012 #elliptical 
gripper.gripper_action(int(func_from_distance_to_int(grasp_width)))
time.sleep(0.1)
palm.goto((0,150))
### lift up
lift_pose = ur.getl()
lift_pose[2] = lift_pose[2]+0.07
ur.movel(lift_pose, tcp_acc, tcp_vel)
### open gripper
palm.goto((-10,150))
gripper.gripper_action(int(func_from_distance_to_int(grasp_width+0.015)))
time.sleep(0.5)
### reorient object
normal_reorient = 1
if normal_reorient == 1:
    ### reorient object
    palm.rotate((-10,146), (-15,150), -80, 1)
else: # pivot up and down
    palm.goto((-15,158))
    time.sleep(1)
    palm.goto((-23,159))
    time.sleep(1)
    palm.goto((-10,150))
    time.sleep(1)
    palm.goto((-15,140))
    time.sleep(1)
#     new_pos = palm.rotate((-5,150), (-10,150), 100, 0.5)
#     time.sleep(0.5)
#     new_pos = palm.rotate(new_pos, (-10,150), -150, 0.5)
time.sleep(1)
### close gripper
palm.goto((-10,130))
gripper.gripper_action(int(func_from_distance_to_int(grasp_width-0.02)))
# palm.goto((0,110))

Rotated to: (-18.039695639734642, 144.36437666111564)


#### release single object

In [24]:
### open gripper and ready for reorient
palm.goto((0,130))
time.sleep(0.5)
palm.goto((-10,150))
time.sleep(0.5)
gripper.gripper_action(int(func_from_distance_to_int(grasp_width+0.01)))
time.sleep(0.5)

### reorient
palm.rotate((-10,150), (-5,150), 80, 1)
time.sleep(1.5)

### close gripper
palm.goto((-5,145))
gripper.gripper_action(int(func_from_distance_to_int(grasp_width-0.004)))

### lower robot
drop_pose = ur.getl()
drop_pose[1] = drop_pose[1]-0.0015
drop_pose[2] = drop_pose[2]-0.075
ur.movel(drop_pose, tcp_acc, tcp_vel)

### release object
palm.goto((13,150))
gripper.gripper_action(int(func_from_distance_to_int(center_width+0.02))) 

lift_pose = ur.getl()
lift_pose[2] = lift_pose[2]+0.07
ur.movel(lift_pose, tcp_acc, tcp_vel)

-5.8965368076764095 145.08103448350352
Rotated to: (-5.8965368076764095, 145.08103448350352)


<Transform:
<Orientation: 
array([[ 9.99999991e-01, -6.63155641e-05,  1.17606562e-04],
       [-6.63346776e-05, -9.99999985e-01,  1.62524735e-04],
       [ 1.17595783e-04, -1.62532535e-04, -9.99999980e-01]])>
<Vector: (-0.08983, 0.48509, 0.07016)>
>

#### pick next object

In [51]:
gripper.gripper_action(int(func_from_distance_to_int(grasp_width-0.01)))
palm.goto((-20,130))
time.sleep(1)
palm.goto((-20,140))
time.sleep(1)
palm.goto((15,140))
time.sleep(1)

gripper.gripper_action(int(func_from_distance_to_int(center_width+0.02))) 
palm.goto((15,150))
time.sleep(0.5)

ur.movel((-0.09, 0.485, 0.005, pi, 0, 0), tcp_acc, tcp_vel)
time.sleep(0.5)

grasp_width = center_width+0.012 #elliptical 
gripper.gripper_action(int(func_from_distance_to_int(grasp_width)))
time.sleep(0.1)
palm.goto((0,150))
time.sleep(0.5)

lift_pose = ur.getl()
lift_pose[2] = lift_pose[2]+0.07
ur.movel(lift_pose, tcp_acc, tcp_vel)
time.sleep(0.5)

palm.goto((-10,150))
gripper.gripper_action(int(func_from_distance_to_int(grasp_width+0.010)))
time.sleep(1)

palm.rotate((-10,146), (-15,150), -80, 1)
time.sleep(1)

palm.goto((-10,130))
gripper.gripper_action(int(func_from_distance_to_int(grasp_width-0.02)))
time.sleep(1)

Rotated to: (-18.05674455264937, 144.37360570704038)


In [74]:
palm.disable_motor()

disabling motor control...


In [75]:
palm.enable_motor()

enabling motor control...


In [63]:
palm.home()

homing...
enabling motor control...


In [30]:
palm.get_pos()

(0.0815746470001075, 77.72131715725102)

In [62]:
palm.set_soft()

In [57]:
palm.set_hard()

In [76]:
for i in range(3):
    palm.goto((-30,70))
    time.sleep(0.07)
    palm.goto((30,70))
    time.sleep(0.07)
    palm.goto((0,150))
    time.sleep(0.07)